# Microsoft Fabric Dataflow Incremental Refresh Notebook

## Prerequisites

Before using this notebook, ensure you have the following:

### Required Access & Permissions
- **Microsoft Fabric Workspace**: Contributor or Admin role in the target workspace
- **Warehouse Permissions**: Read/Write access to the warehouse where tracking tables will be created
- **Dataflow Permissions**: Permission to trigger and monitor dataflow refreshes

### Technical Requirements
- **Microsoft Fabric Capacity**: Active Fabric capacity (F2 or higher recommended)
- **Python Environment**: Fabric notebook environment with the following packages:
  - `pandas` - Data manipulation
  - `sempy.fabric` - Microsoft Fabric API client
  - `notebookutils.data` - Fabric data connection utilities
- **Dataflow Gen2**: Existing dataflow configured in Microsoft Fabric

### Knowledge Prerequisites
- Basic understanding of Microsoft Fabric dataflows
- Familiarity with incremental refresh concepts
- SQL query knowledge for troubleshooting
- Understanding of Power Query M language for dataflow integration

## Overview

This notebook implements an advanced framework for incremental data refresh in Microsoft Fabric dataflows. It is designed to handle scenarios where standard incremental refresh does not work due to limitations in the data source, such as query folding issues or bucket size constraints.

The notebook supports both **regular Dataflow Gen2** and **CI/CD Dataflow Gen2** objects, automatically detecting the dataflow type and using the appropriate Microsoft Fabric REST API endpoints.

## Architecture

The notebook orchestrates a coordinated refresh process across multiple Fabric components:

```
┌─────────────┐
│   Pipeline  │ (Triggers notebook with parameters)
└──────┬──────┘
       │
       ▼
┌─────────────────────────────────────────────────────────┐
│              Notebook (DataflowRefresher)               │
│  ┌────────────────────────────────────────────────────┐ │
│  │ 1. Read tracking table from Warehouse              │ │
│  │ 2. Calculate date ranges & buckets                 │ │
│  │ 3. Update tracking table (Running status)          │ │
│  │ 4. Call Fabric REST API to trigger dataflow        │ │
│  │ 5. Poll for completion status                      │ │
│  │ 6. Update tracking table (Success/Failed status)   │ │
│  └────────────────────────────────────────────────────┘ │
└─────────┬───────────────────────────────────┬───────────┘
          │                                   │
          ▼                                   ▼
┌─────────────────────┐           ┌─────────────────────┐
│   Warehouse         │           │  Fabric REST API    │
│  [Incremental       │           │  - Regular DF:      │
│   Update] Table     │           │    /dataflows/...   │
│  - range_start      │           │  - CI/CD DF:        │
│  - range_end        │           │    /items/.../jobs  │
│  - status           │           └──────────┬──────────┘
└─────────┬───────────┘                      │
          │                                  ▼
          │                        ┌───────────────────┐
          └───────────────────────>│   Dataflow Gen2   │
            (Dataflow reads range) │  (Power Query M)  │
                                   └─────────┬─────────┘
                                             │
                                             ▼
                                   ┌───────────────────┐
                                   │  Data Source      │
                                   │  (Filtered by     │
                                   │   date range)     │
                                   └─────────┬─────────┘
                                             │
                                             ▼
                                   ┌───────────────────┐
                                   │   Warehouse       │
                                   │   Destination     │
                                   │   Table           │
                                   └───────────────────┘
```

**Key Flow:**
1. **Pipeline** passes parameters to notebook
2. **Notebook** manages the tracking table and orchestrates refresh
3. **Tracking table** stores date ranges that the dataflow reads
4. **Dataflow** executes with filtered date range from tracking table
5. **Data** flows from source to warehouse destination table

## Key Features

- **Intelligent Bucket Processing**: Splits large date ranges into configurable buckets to manage incremental loads efficiently
- **Automatic Retry Mechanism**: Retries failed bucket refreshes with exponential backoff before failing the entire process
- **Dataflow Type Auto-Detection**: Automatically detects and supports both regular and CI/CD dataflows
- **Connection Management**: Handles database connections with automatic retry and reconnection logic to prevent timeout issues
- **Comprehensive Status Tracking**: Maintains detailed metadata about refresh operations in a warehouse tracking table
- **Pipeline Failure Integration**: Exits with proper failure codes to integrate with Fabric pipeline error handling

## Pipeline Parameters

Configure the following parameters when setting up the notebook activity in your Fabric pipeline:

| Parameter | Type | Required | Description |
|-----------|------|----------|-------------|
| `workspace_id` | String | Yes | ID of the Fabric workspace containing the dataflow |
| `dataflow_id` | String | Yes | ID of the dataflow to refresh |
| `dataflow_name` | String | Yes | Name or description of the dataflow |
| `initial_load_from_date` | String | Yes* | Start date for initial historical load (format: 'YYYY-MM-DD'). *Required only for first load |
| `bucket_size_in_days` | Integer | No | Size of each refresh bucket in days (default: 1) |
| `bucket_retry_attempts` | Integer | No | Number of retry attempts for failed buckets (default: 3) |
| `incrementally_update_last_n_days` | Integer | No | Number of days to overlap/refresh in incremental updates (default: 1) |
| `reinitialize_dataflow` | Boolean | No | Set to True to delete tracking data and restart from scratch (default: False) |
| `destination_table` | String | Yes | Name of the destination table in the warehouse where data is written. Can be just table name (uses dbo schema) or `schema.table` format for tables in other schemas |
| `incremental_update_column` | String | Yes | DateTime column used for incremental filtering |
| `is_cicd_dataflow` | Boolean | No | Explicitly specify if this is a CI/CD dataflow (auto-detected if not provided) |

## Notebook Constants

The following constants must be configured inside the notebook:

| Constant | Example | Description |
|----------|---------|-------------|
| `SCHEMA` | `"[Warehouse DB].[dbo]"` | Database schema where the tracking table resides |
| `INCREMENTAL_TABLE` | `"[Incremental Update]"` | Name of the metadata tracking table |
| `CONNECTION_ARTIFACT` | `"Warehouse name or id"` | Name of the warehouse artifact |
| `CONNECTION_ARTIFACT_ID` | `"Workspace id"` | Technical ID of the warehouse |
| `CONNECTION_ARTIFACT_TYPE` | `"Warehouse"` | Type of artifact (typically "Warehouse") |

## Setup Instructions

Follow these steps to set up and configure the notebook:

### Step 1: Import the Notebook
1. Navigate to your Microsoft Fabric workspace
2. Click **New** → **Import notebook**
3. Upload the `Incrementally Refresh Dataflow.ipynb` file
4. Wait for the import to complete

### Step 2: Configure Notebook Constants
Open the notebook and update the following constants in the third code cell:

```python
# Update these constants with your warehouse details
SCHEMA = "[YourWarehouseName].[dbo]"
INCREMENTAL_TABLE = "[Incremental Update]"
CONNECTION_ARTIFACT = "YourWarehouseName"
CONNECTION_ARTIFACT_ID = "your-warehouse-id-guid"
CONNECTION_ARTIFACT_TYPE = "Warehouse"
```

**How to find your Warehouse ID:**
1. Open your warehouse in Fabric
2. Check the URL: `https://app.fabric.microsoft.com/groups/{workspace_id}/warehouses/{warehouse_id}`
3. Copy the `warehouse_id` GUID from the URL

### Step 3: Create a Fabric Pipeline
1. In your workspace, create a new **Data Pipeline**
2. Add a **Notebook** activity to the pipeline canvas
3. Configure the notebook activity:
   - **Notebook**: Select the imported notebook
   - **Parameters**: Add the required parameters (see Quick Start Example below)

### Step 4: Configure Your Dataflow
Ensure your dataflow Power Query includes logic to read `range_start` and `range_end` from the tracking table (see Integration section).

### Step 5: Test the Setup
1. Run the pipeline with all required parameters
2. Monitor the notebook execution in Fabric
3. Check the `[Incremental Update]` table in your warehouse to verify tracking records are created
4. Verify data appears in your destination table

## Quick Start Example

Here's a complete example of how to configure the pipeline parameters for your first run:

### Example Configuration

**Pipeline Parameters:**
```json
{
  "workspace_id": "a1b2c3d4-e5f6-7890-abcd-ef1234567890",
  "dataflow_id": "x9y8z7w6-v5u4-3210-zyxw-vu9876543210",
  "dataflow_name": "Sales Data Incremental Refresh",
  "initial_load_from_date": "2024-01-01",
  "bucket_size_in_days": 7,
  "bucket_retry_attempts": 3,
  "incrementally_update_last_n_days": 2,
  "reinitialize_dataflow": false,
  "destination_table": "FactSales",
  "incremental_update_column": "OrderDate",
  "is_cicd_dataflow": null
}
```

**Parameter Explanation:**
- `workspace_id`: Your Fabric workspace GUID (found in workspace URL)
- `dataflow_id`: Your dataflow GUID (found in dataflow URL)
- `dataflow_name`: Descriptive name for logging/tracking
- `initial_load_from_date`: Start loading data from January 1, 2024 (first run only)
- `bucket_size_in_days`: Process 7 days at a time
- `incrementally_update_last_n_days`: Overlap last 2 days on each refresh
- `destination_table`: Table name in warehouse (uses `dbo` schema by default)
- `incremental_update_column`: Date column used for filtering
- `is_cicd_dataflow`: Auto-detect dataflow type (set to `true` or `false` to override)

### Expected First Run Behavior

1. **No tracking record exists** → Initial load scenario
2. Calculates range: `2024-01-01` to `yesterday 23:59:59`
3. Splits into 7-day buckets
4. Processes each bucket sequentially
5. Creates tracking table entry
6. Dataflow reads `range_start` and `range_end` from tracking table
7. Data is loaded into `dbo.FactSales` table

### Expected Subsequent Runs

1. **Tracking record exists** → Incremental update scenario
2. Reads last successful range end date
3. Calculates new range with 2-day overlap
4. Processes new buckets
5. Updates tracking table with new range

## How It Works

### 1. Metadata Table Management

The notebook automatically creates and manages an `[Incremental Update]` tracking table in the warehouse with the following schema:

- `dataflow_id`, `workspace_id`, `dataflow_name`: Dataflow identifiers
- `initial_load_from_date`: Historical start date
- `bucket_size_in_days`, `incrementally_update_last_n_days`: Configuration parameters
- `destination_table`, `incremental_update_column`: Target table information
- `update_time`, `status`: Current refresh status and timestamp
- `range_start`, `range_end`: Date range for the current/last refresh bucket
- `is_cicd_dataflow`: Flag indicating dataflow type (for API routing)

### 2. Dataflow Type Detection

The notebook automatically detects whether the dataflow is a CI/CD or regular dataflow by probing the official Microsoft Fabric API endpoints:

- **CI/CD Dataflows**: Uses `/v1/workspaces/{workspace_id}/items/{dataflow_id}/jobs/instances` endpoint
- **Regular Dataflows**: Uses `/v1.0/myorg/groups/{workspace_id}/dataflows/{dataflow_id}/refreshes` endpoint

### 3. Processing Logic

#### **Scenario A: Initial Load (No Previous Refresh)**

1. Validates that `initial_load_from_date` is provided
2. Calculates date range from `initial_load_from_date` to yesterday at 23:59:59
3. Splits the date range into buckets based on `bucket_size_in_days`
4. For each bucket:
   - Deletes any overlapping data in the destination table
   - Updates tracking table with "Running" status
   - Triggers dataflow refresh
   - Waits for completion and monitors status
   - **If bucket fails**: Retries up to `bucket_retry_attempts` times with exponential backoff (30s, 60s, 120s, etc.)
   - **If all retries fail**: Logs error, updates tracking table, and **exits with failure code (1)** to fail the pipeline
   - **If bucket succeeds**: Moves to next bucket

#### **Scenario B: Previous Refresh Failed**

1. Detects failed status from previous run
2. Retrieves the failed bucket's date range
3. Retries the failed bucket using the same retry logic as above
4. **If all retries fail**: Exits with failure code to fail the pipeline
5. **If retry succeeds**: Continues with normal incremental processing

#### **Scenario C: Incremental Update (Previous Refresh Successful)**

1. Calculates new date range:
   - If `incrementally_update_last_n_days` is set: Uses `min(last_end_date + 1 second, yesterday - N days)` to ensure overlap without gaps
   - Otherwise: Starts from `last_end_date + 1 second`
   - End date is always yesterday at 23:59:59
2. Splits date range into buckets if needed
3. Processes each bucket with the same retry logic as initial load
4. **If any bucket fails after all retries**: Exits with failure code to fail the pipeline

### 4. Retry Mechanism with Exponential Backoff

When a bucket refresh fails, the notebook:

1. **Retry 1**: Waits 30 seconds, then retries
2. **Retry 2**: Waits 60 seconds, then retries
3. **Retry 3**: Waits 120 seconds, then retries
4. **If all retries fail**:
   - Updates tracking table with failed status
   - Logs detailed error message
   - Raises `RuntimeError` with failure details
   - **Exits with `sys.exit(1)`** to mark the notebook as failed in the pipeline
   - **No further buckets are processed**

This ensures transient issues (network glitches, temporary service unavailability) are handled gracefully, while persistent failures properly fail the pipeline.

### 5. Date Range Logic

- **End Date**: Always defaults to yesterday at 23:59:59 (never includes today's partial data)
- **Bucket End Times**: Set to 23:59:59 except for the final bucket
- **Next Bucket Start**: Previous bucket end + 1 second (ensures no gaps or overlaps)
- **Overlap Handling**: When `incrementally_update_last_n_days` is set, the start date ensures overlap while avoiding gaps

### 6. Connection Management

The notebook proactively manages database connections to prevent timeout issues:

- Closes connections before long-running dataflow operations
- Validates and recreates connections as needed
- Implements retry logic for all database operations (max 3 retries with 1-second delays)

## Execution Results

Upon completion or failure, the notebook prints:

```
Dataflow refresh execution completed:
Status: Completed / Completed with N failures / Failed: [error message]
Total buckets processed: N
Successful refreshes: N
Failed refreshes: N
Total retry attempts: N
Duration: X.XX seconds
Dataflow type: CI/CD / Regular
```

If a bucket fails after all retries:

```
================================================================================
DATAFLOW REFRESH FAILED
================================================================================
Error: Bucket N/M failed after 3 attempts with status Failed. Range: YYYY-MM-DD to YYYY-MM-DD

The dataflow refresh has been terminated due to bucket failure after all retry attempts.
Please check the logs above for detailed error information.
================================================================================
```

The notebook then exits with code 1, causing the Fabric pipeline to mark the notebook activity as **Failed**.

## Integration with Dataflow Power Query

Your dataflow Power Query should read the `range_start` and `range_end` parameters from the `[Incremental Update]` table:

```powerquery
let
    Source = Sql.Database("[server]", "[database]"),
    TrackingTable = Source{[Schema="dbo", Item="Incremental Update"]}[Data],
    FilteredRows = Table.SelectRows(TrackingTable, each [dataflow_id] = "your-dataflow-id"),
    RangeStart = FilteredRows{0}[range_start],
    RangeEnd = FilteredRows{0}[range_end],

    // Use RangeStart and RangeEnd to filter your data source
    FilteredData = Table.SelectRows(YourDataSource,
        each [YourDateColumn] >= RangeStart and [YourDateColumn] <= RangeEnd)
in
    FilteredData
```

## Best Practices

1. **Bucket Size**: Start with 1 day buckets. Increase if your data volume is low and performance is not a concern.
2. **Retry Attempts**: Default of 3 is recommended. Increase only if you experience frequent transient failures.
3. **Overlap Days**: Set `incrementally_update_last_n_days` to 1 or more if your source data can be updated retroactively.
4. **Destination Table Schema**:
   - If your table is in the `dbo` schema: Use just the table name (e.g., `"SalesData"`)
   - If your table is in another schema: Use `schema.table` format (e.g., `"staging.SalesData"` or `"analytics.SalesData"`)
5. **Monitoring**: Monitor the `[Incremental Update]` table in your warehouse to track refresh history and troubleshoot issues.
6. **Pipeline Design**: Use the notebook activity failure to trigger alerts or retry logic at the pipeline level.

## Troubleshooting

### Notebook fails immediately

**Symptoms:**
- Notebook activity fails within seconds
- Error: "Missing required parameter" or "Connection failed"

**Solutions:**
1. **Verify all required parameters are provided:**
   - `workspace_id`, `dataflow_id`, `dataflow_name`
   - `destination_table`, `incremental_update_column`
   - `initial_load_from_date` (required for first run only)

2. **Check warehouse connection:**
   ```sql
   -- Test warehouse connection by running this in your warehouse
   SELECT TOP 1 * FROM INFORMATION_SCHEMA.TABLES
   ```

3. **Verify notebook constants are configured:**
   - Open the notebook
   - Check the third code cell for `SCHEMA`, `CONNECTION_ARTIFACT_ID`, etc.
   - Ensure the warehouse ID is correct (copy from warehouse URL)

**Common Errors:**
- `"initial_load_from_date is required for the first load"` → Add this parameter for first execution
- `"Connection timeout"` → Verify warehouse is running and accessible
- `"Table does not exist"` → Notebook will auto-create tracking table on first run

### Bucket keeps failing after retries

**Symptoms:**
- Individual buckets fail repeatedly
- Error: "Bucket N/M failed after 3 attempts with status Failed"
- Pipeline marked as failed

**Solutions:**
1. **Check dataflow execution logs:**
   - Open the dataflow in Fabric
   - Navigate to **Refresh history**
   - Review error messages from failed refreshes

2. **Verify Power Query configuration:**
   - Ensure dataflow reads `range_start` and `range_end` correctly
   - Test with a small date range manually
   - Check for data type mismatches

3. **Inspect the tracking table:**
   ```sql
   -- Check current tracking state
   SELECT TOP 5
       dataflow_id,
       dataflow_name,
       status,
       range_start,
       range_end,
       update_time
   FROM [dbo].[Incremental Update]
   WHERE dataflow_id = 'your-dataflow-id'
   ORDER BY update_time DESC
   ```

4. **Reduce bucket size:**
   - Try smaller `bucket_size_in_days` (e.g., 1 day instead of 7)
   - Large date ranges may timeout or exceed memory limits

5. **Check data source:**
   - Verify source system is accessible
   - Check for connectivity issues during refresh window
   - Confirm source data exists for the date range

### Wrong dataflow type detected

**Symptoms:**
- Error: "Dataflow refresh failed with status 404" or "Endpoint not found"
- Notebook detects wrong dataflow type (CI/CD vs Regular)

**Solutions:**
1. **Explicitly set the dataflow type:**
   ```json
   {
     "is_cicd_dataflow": true   // or false for regular dataflows
   }
   ```

2. **Verify workspace and dataflow IDs:**
   - Check the URL when viewing your dataflow
   - Ensure no typos in the GUID values
   - Confirm the dataflow exists in the specified workspace

3. **Check dataflow type in Fabric:**
   - CI/CD dataflows are typically created through Git integration
   - Regular dataflows are created directly in the workspace

### Database connection timeouts

**Symptoms:**
- Error: "Connection lost" or "Idle timeout"
- Random failures during long-running refreshes

**Solutions:**
- The notebook handles this automatically with retry logic and connection management
- Connections are closed before long dataflow operations
- If persistent, check:
  - Warehouse availability and health status
  - Network connectivity between notebook and warehouse
  - Fabric capacity resource limits

### Tracking table issues

**Problem: Tracking table shows "Running" but notebook completed**

**Solution:**
```sql
-- Manually check for stuck records
SELECT * FROM [dbo].[Incremental Update]
WHERE status = 'Running'
  AND update_time < DATEADD(HOUR, -2, GETDATE())

-- If needed, manually update stuck records
UPDATE [dbo].[Incremental Update]
SET status = 'Failed'
WHERE dataflow_id = 'your-dataflow-id'
  AND status = 'Running'
  AND update_time < DATEADD(HOUR, -2, GETDATE())
```

**Problem: Need to restart from scratch**

**Solution:**
```sql
-- Option 1: Delete tracking record (will trigger initial load on next run)
DELETE FROM [dbo].[Incremental Update]
WHERE dataflow_id = 'your-dataflow-id'

-- Option 2: Use reinitialize_dataflow parameter
-- Set reinitialize_dataflow = true in pipeline parameters
```

### Monitoring and debugging

**Check tracking table history:**
```sql
-- View refresh history
SELECT
    dataflow_name,
    status,
    range_start,
    range_end,
    update_time,
    DATEDIFF(SECOND,
        LAG(update_time) OVER (PARTITION BY dataflow_id ORDER BY update_time),
        update_time
    ) as seconds_since_last_refresh
FROM [dbo].[Incremental Update]
WHERE dataflow_id = 'your-dataflow-id'
ORDER BY update_time DESC
```

**Check for data gaps:**
```sql
-- Identify gaps in refreshed date ranges
WITH RangedData AS (
    SELECT
        range_start,
        range_end,
        LEAD(range_start) OVER (ORDER BY range_start) as next_start
    FROM [dbo].[Incremental Update]
    WHERE dataflow_id = 'your-dataflow-id'
      AND status = 'Success'
)
SELECT
    range_end as gap_start,
    next_start as gap_end,
    DATEDIFF(DAY, range_end, next_start) as gap_days
FROM RangedData
WHERE DATEADD(SECOND, 1, range_end) < next_start
```

### Getting help

If you continue experiencing issues:
1. Check the notebook execution logs in Fabric
2. Review the dataflow refresh history
3. Verify all configuration values match your environment
4. Test with a minimal date range (1-2 days) first

## Version History

- **v3.0**: Added bucket retry mechanism with exponential backoff and pipeline failure integration
- **v2.0**: Added CI/CD dataflow support with automatic type detection
- **v1.0**: Initial implementation with basic incremental refresh framework

In [ ]:
# Parameters passed from the pipeline as base parameters of the notebook activity
# Workspace id
workspace_id = ''

# Dataflow id
dataflow_id = ''

# Dataflow name
dataflow_name = ''

# Initial load from date for the first load
initial_load_from_date = ''

# Bucket size for each load
bucket_size_in_days = 1

# Number of retry attempts for failed bucket refreshes
bucket_retry_attempts = 3

# Reinitialize dataflow
reinitialize_dataflow = False

# Incrementally update last n days
incrementally_update_last_n_days = 1

# Destination table
destination_table = ''

# Incremental update column
incremental_update_column = ''

# Indicates if this is a CI/CD dataflow (optional, auto-detected if not specified)
is_cicd_dataflow = None

In [ ]:
from datetime import datetime, timedelta
import time
import logging
import pandas as pd
import sempy.fabric as fabric
from typing import Optional, Dict, Any, Union, Tuple, List

# Constants
SCHEMA = "[Warehouse DB].[dbo]"
INCREMENTAL_TABLE = "[Incremental Update]"
CONNECTION_ARTIFACT = "Warehouse name or id"
CONNECTION_ARTIFACT_ID = "Workspace id"
CONNECTION_ARTIFACT_TYPE = "Warehouse"

class DataflowRefresher:
    """
    Class to incrementally refresh a dataflow in Microsoft Fabric
    with support for initial loads and incremental updates
    Enhanced to support both regular dataflow gen2 and dataflow gen2 CI/CD objects
    using official Microsoft Fabric APIs
    """
    def __init__(self, client, artifact: str, artifact_id: str, artifact_type: str,
                 schema: str, incremental_table: str, log_level: int):
        """
        Initialize the DataflowRefresher
        
        Args:
            client: API client for Microsoft Fabric
            artifact: Name of the artifact to connect to (e.g., "The Beer Store")
            artifact_id: ID of the artifact (e.g., workspace ID)
            artifact_type: Type of the artifact (e.g., "Warehouse")
            schema: Database schema name including brackets, e.g. "[The Beer Store].[dbo]"
            incremental_table: Name of the table tracking incremental updates (without schema)
            log_level: level of logging
        """
        self.client = client
        self.artifact = artifact
        self.artifact_id = artifact_id
        self.artifact_type = artifact_type
        self.connection = None
        self.schema = schema
        self.incremental_table = f"{schema}.{incremental_table}"
        self.logger = logging.getLogger(__name__)
        self.logger.setLevel(log_level)
        self._ensure_connection()
        self.create_incremental()
    
    def _create_connection(self):
        """
        Create a new connection to the artifact
        
        Returns:
            A new database connection
        """
        try:
            import notebookutils.data
            return notebookutils.data.connect_to_artifact(
                self.artifact, self.artifact_id, self.artifact_type)
        except Exception as e:
            self.logger.error(f"Error creating connection: {e}")
            raise
    
    def _ensure_connection(self):
        """
        Ensure that we have a valid database connection, creating a new one if needed
        
        Returns:
            A valid database connection
        """
        if self.connection is None:
            self.connection = self._create_connection()
            return self.connection
            
        # Test if the existing connection is still valid
        try:
            cursor = self.connection.cursor()
            cursor.execute("SELECT 1")
            cursor.fetchall()
            cursor.close()
            return self.connection
        except Exception as e:
            self.logger.warning(f"Connection test failed, reconnecting: {e}")
            self._close_connection()
            self.connection = self._create_connection()
            return self.connection
    
    def _close_connection(self):
        """Close the current connection if it exists"""
        if self.connection is not None:
            try:
                self.connection.close()
            except Exception as e:
                self.logger.warning(f"Error closing connection: {e}")
            finally:
                self.connection = None
    
    def _execute_with_retry(self, sql, params=None, commit=True, max_retries=3):
        """
        Execute a SQL statement with automatic retry on connection issues
        
        Args:
            sql: SQL statement to execute
            params: Parameters for the SQL statement
            commit: Whether to commit the transaction
            max_retries: Maximum number of retries
            
        Returns:
            Database cursor
        """
        retries = 0
        last_error = None
        
        while retries < max_retries:
            try:
                # Ensure we have a valid connection
                self._ensure_connection()
                
                # Execute the SQL
                cursor = self.connection.execute(sql, params or ())
                
                # Commit if requested
                if commit:
                    self.connection.commit()
                    
                return cursor
                
            except Exception as e:
                last_error = e
                self.logger.warning(f"Database operation failed (attempt {retries+1}/{max_retries}): {e}")
                self._close_connection()  # Force reconnection on next attempt
                retries += 1
                
                # Small delay before retry
                if retries < max_retries:
                    time.sleep(1)
        
        # If we get here, we've exhausted retries
        self.logger.error(f"Database operation failed after {max_retries} attempts: {last_error}")
        raise last_error
    
    def create_incremental(self) -> bool:
        """
        Create the incremental update meta data table if it does not exist
        
        Args:
            None
            
        Returns:
            None
        """
        try:
            sql = f"""
                IF NOT EXISTS (SELECT * FROM sys.tables WHERE name = 'Incremental Update' AND schema_id = SCHEMA_ID('dbo'))
                BEGIN
                CREATE TABLE {self.incremental_table}
                (
                    [dataflow_id] [varchar](60) NOT NULL,
                    [workspace_id] [VARCHAR](60) NOT NULL,
                    [dataflow_name] [varchar](60) NULL,
                    [initial_load_from_date] [datetime2](3) NOT NULL,
                    [bucket_size_in_days] [int] NOT NULL,
                    [incrementally_update_last_n_days] [int] NOT NULL,
                    [destination_table] [VARCHAR](60) NOT NULL,
                    [incremental_update_column] [VARCHAR](60) NOT NULL,
                    [update_time] [datetime2](3) NULL,
                    [status] [varchar](50) NOT NULL,
                    [range_start] [datetime2](3) NULL,
                    [range_end] [datetime2](3) NULL,
                    [is_cicd_dataflow] [bit] NULL
                )
                END
                ELSE
                BEGIN
                    -- Add the new column if it doesn't exist (for backward compatibility)
                    IF NOT EXISTS (SELECT * FROM sys.columns WHERE object_id = OBJECT_ID('{self.incremental_table}') AND name = 'is_cicd_dataflow')
                    BEGIN
                        ALTER TABLE {self.incremental_table} ADD [is_cicd_dataflow] [bit] NULL
                    END
                END
            """
            self._execute_with_retry(sql)
            self.logger.info(f"Successfully created/updated incremental update table in the database")
            return True
            
        except Exception as e:
            self.logger.error(f"Error creating/updating incremental refresh table: {e}")
            raise

    def _detect_dataflow_type(self, workspace_id: str, dataflow_id: str) -> bool:
        """
        Auto-detect if a dataflow is a CI/CD dataflow by checking the official endpoints
        
        Args:
            workspace_id: ID of the workspace
            dataflow_id: ID of the dataflow
            
        Returns:
            True if CI/CD dataflow, False if regular dataflow
        """
        try:
            # Try the official CI/CD endpoint first
            try:
                endpoint = f"/v1/workspaces/{workspace_id}/items/{dataflow_id}"
                response = self.client.get(endpoint)
                
                if response.status_code == 200:
                    item_info = response.json()
                    # Check if it's a dataflow and has CI/CD characteristics
                    item_type = item_info.get("type", "").lower()
                    if item_type == "dataflow" or "dataflow" in item_type:
                        self.logger.info(f"Detected CI/CD dataflow for {dataflow_id}")
                        return True
                        
            except Exception as e:
                self.logger.debug(f"CI/CD endpoint check failed: {e}")
            
            # Try regular dataflow endpoint
            try:
                endpoint = f"/v1.0/myorg/groups/{workspace_id}/dataflows/{dataflow_id}"
                response = self.client.get(endpoint)
                
                if response.status_code == 200:
                    self.logger.info(f"Detected regular dataflow for {dataflow_id}")
                    return False
                    
            except Exception as e:
                self.logger.debug(f"Regular dataflow endpoint check failed: {e}")
                
            # Default to regular dataflow if detection fails
            self.logger.warning(f"Could not auto-detect dataflow type for {dataflow_id}, defaulting to regular dataflow")
            return False
            
        except Exception as e:
            self.logger.warning(f"Error during dataflow type detection: {e}")
            return False

    def get_incremental(self, dataflow_id: str) -> Optional[pd.DataFrame]:
        """
        Get the last refresh details from the tracking table
        
        Args:
            dataflow_id: ID of the dataflow
            
        Returns:
            DataFrame with the last refresh details or None if no records found
        """
        try:
            sql = f"""
                SELECT TOP (1) [dataflow_id],
                         [update_time],
                         [status],
                         [range_start],
                         [range_end],
                         [is_cicd_dataflow]
                FROM {self.incremental_table}
                WHERE [dataflow_id] = ?
                ORDER BY [update_time] DESC
            """
            cursor = self._execute_with_retry(sql, (dataflow_id,), commit=False)
            columns = [column[0] for column in cursor.description]
            data = cursor.fetchall()
            data = [tuple(row) for row in data]
            
            if not data:
                self.logger.info(f"No previous refresh records found for dataflow {dataflow_id}")
                return None
                
            return pd.DataFrame(data, columns=columns)
            
        except Exception as e:
            self.logger.error(f"Error getting incremental refresh data: {e}")
            raise

    def insert_into_incremental(self, dataflow_id: str, workspace_id: str, dataflow_name: str, initial_load_from_date: str,
                            bucket_size_in_days: int, incrementally_update_last_n_days: int, destination_table: str,
                            incremental_update_column: str, status: str, 
                            range_start: datetime, range_end: datetime, is_cicd_dataflow: bool = False) -> bool:
        """
        Insert into incremental table in the warehouse
        
        Args:
            dataflow_id: ID of the dataflow
            workspace_id: ID of the workspace
            dataflow_name: Name or description of the dataflow
            initial_load_from_date: Initial load from date
            bucket_size_in_days: Bucket size of each refresh in days
            incrementally_update_last_n_days: Incremental refresh bucket size
            destination_table: Destination table in the warehouse where data is written
            incremental_update_column: Column of the destination table used for incremental update
            status: Status of the refresh operation
            range_start: Start of the refresh date range
            range_end: End of the refresh date range
            is_cicd_dataflow: Whether this is a CI/CD dataflow
            
        Returns:
            True if successful, False otherwise
        """
        try:
            current_time = datetime.now().strftime('%Y-%m-%d %H:%M:%S.%f')[:-3]
            start_formatted = range_start.strftime('%Y-%m-%d %H:%M:%S.%f')[:-3]
            end_formatted = range_end.strftime('%Y-%m-%d %H:%M:%S.%f')[:-3]
            
            sql = f"""
                INSERT INTO {self.incremental_table}
                (
                    [dataflow_id], 
                    [workspace_id],
                    [dataflow_name],
                    [initial_load_from_date],
                    [bucket_size_in_days],
                    [incrementally_update_last_n_days],
                    [destination_table],
                    [incremental_update_column],
                    [update_time],
                    [status],
                    [range_start],
                    [range_end],
                    [is_cicd_dataflow]
                )
                VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
            """
            
            params = (dataflow_id, workspace_id, dataflow_name, initial_load_from_date, 
                     bucket_size_in_days, incrementally_update_last_n_days, destination_table, 
                     incremental_update_column, current_time, status, start_formatted, end_formatted, is_cicd_dataflow)
            
            self._execute_with_retry(sql, params)
            self.logger.info(f"Successfully inserted record for dataflow {dataflow_id}")
            return True
            
        except Exception as e:
            self.logger.error(f"Error inserting into incremental table: {e}")
            raise

    def delete_incremental(self, dataflow_id: str) -> bool:
        """
        Delete the entry from the tracking table
        
        Args:
            dataflow_id: ID of the dataflow
            
        Returns:
            True if successful, False otherwise
        """
        try:
            sql = f"""
                DELETE
                FROM {self.incremental_table}
                WHERE [dataflow_id] = ?
            """
            self._execute_with_retry(sql, (dataflow_id,))
            self.logger.info(f"Successfully deleted old record for dataflow {dataflow_id}")
            return True
            
        except Exception as e:
            self.logger.error(f"Could not delete old record for dataflow: {e}")
            raise

    def update_incremental(self, dataflow_id: str, status: str, 
                        range_start: datetime, range_end: datetime) -> bool:
        """
        Update incremental table in the warehouse with the new parameters
        
        Args:
            dataflow_id: ID of the dataflow
            status: Status of the refresh operation
            range_start: Start of the refresh date range
            range_end: End of the refresh date range
            
        Returns:
            True if successful, False otherwise
        """
        try:
            current_time = datetime.now().strftime('%Y-%m-%d %H:%M:%S.%f')[:-3]
            start_formatted = range_start.strftime('%Y-%m-%d %H:%M:%S.%f')[:-3]
            end_formatted = range_end.strftime('%Y-%m-%d %H:%M:%S.%f')[:-3]
            
            sql = f"""
                UPDATE {self.incremental_table}
                SET 
                [status] = ?,
                [update_time] = ?,
                [range_start] = ?,
                [range_end] = ?
                WHERE [dataflow_id] = ?
            """
            
            cursor = self._execute_with_retry(sql, (status, current_time, 
                                               start_formatted, end_formatted, dataflow_id))
            rows_affected = cursor.rowcount
            
            if rows_affected > 0:
                self.logger.info(f"Successfully updated record for dataflow {dataflow_id}")
                return True
            else:
                self.logger.warning(f"No records updated for dataflow {dataflow_id}, record may not exist")
                return False
                
        except Exception as e:
            self.logger.error(f"Error updating incremental table: {e}")
            raise

    def delete_data(self, table: str, column: str,
                 range_start: Union[datetime, str],
                 range_end: Union[datetime, str],
                 dataflow_id: Optional[str] = None) -> int:
        """
        Delete any overlapping data from the previous refreshes

        Args:
            table: Target table name (can be schema.table or just table, defaults to dbo schema)
            column: Date column to use for filtering
            range_start: Start of the date range to delete (datetime or string)
            range_end: End of the date range to delete (datetime or string)
            dataflow_id: Optional ID of the dataflow for logging purposes

        Returns:
            Number of rows deleted
        """
        try:
            # Format dates if they're datetime objects
            start_formatted = range_start.strftime('%Y-%m-%d %H:%M:%S.%f')[:-3] if isinstance(range_start, datetime) else range_start
            end_formatted = range_end.strftime('%Y-%m-%d %H:%M:%S.%f')[:-3] if isinstance(range_end, datetime) else range_end

            # Parse schema and table name
            if '.' in table:
                # Schema.Table format provided
                schema_name, table_name = table.split('.', 1)
                full_table_name = f"[The Beer Store].[{schema_name}].[{table_name}]"
            else:
                # Only table name provided, use default dbo schema
                full_table_name = f"{self.schema}.[{table}]"

            sql = f"""
                DELETE FROM {full_table_name}
                WHERE [{column}] BETWEEN ? AND ?
            """

            cursor = self._execute_with_retry(sql, (start_formatted, end_formatted))
            rows_deleted = cursor.rowcount

            log_id = f" for {dataflow_id}" if dataflow_id else ""
            self.logger.info(f"Successfully deleted {rows_deleted} overlapping records{log_id} in table {table} between {start_formatted} and {end_formatted}")

            return rows_deleted

        except Exception as e:
            self.logger.error(f"Error deleting overlapping records: {e}")
            raise

    def refresh_dataflow(self, workspace_id: str, dataflow_id: str, dataflow_name: str,
                        is_cicd_dataflow: bool = None) -> Dict[str, Any]:
        """
        Refresh dataflow and return the response
        Enhanced to support both regular and CI/CD dataflows using official Microsoft APIs
        
        Args:
            workspace_id: ID of the workspace
            dataflow_id: ID of the dataflow
            dataflow_name: Name of the dataflow
            is_cicd_dataflow: Whether this is a CI/CD dataflow (auto-detected if None)
            
        Returns:
            Dictionary containing the response from the refresh API
        """
        try:
            # Auto-detect if not specified
            if is_cicd_dataflow is None:
                is_cicd_dataflow = self._detect_dataflow_type(workspace_id, dataflow_id)
            
            if is_cicd_dataflow:
                # Official Microsoft Fabric API for CI/CD dataflows - simplified without user parameters
                endpoint = f"/v1/workspaces/{workspace_id}/items/{dataflow_id}/jobs/instances?jobType=Refresh"
                payload = {
                    "executionData": {
                        "DataflowName": dataflow_name
                    }
                }
                
                self.logger.info(f"Using official CI/CD endpoint for dataflow {dataflow_id}")
                
                try:
                    response = self.client.post(endpoint, json=payload)
                    self.logger.info(f"Successfully triggered CI/CD refresh for dataflow {dataflow_id}")
                    return response
                except Exception as e:
                    self.logger.warning(f"Official CI/CD endpoint failed, trying fallback: {e}")
                    # Fallback to regular dataflow endpoint
                    is_cicd_dataflow = False
            
            if not is_cicd_dataflow:
                # Regular dataflow gen2 API
                endpoint = f"/v1.0/myorg/groups/{workspace_id}/dataflows/{dataflow_id}/refreshes"
                payload = {"refreshRequest": "y"}
                
                response = self.client.post(endpoint, json=payload)
                self.logger.info(f"Successfully triggered regular refresh for dataflow {dataflow_id}")
                return response
            
        except Exception as e:
            self.logger.error(f"Error triggering refresh: {e}")
            raise

    def get_latest_refresh_status(self, workspace_id: str, dataflow_id: str, is_cicd_dataflow: bool = None) -> Optional[str]:
        """
        Get refresh status of a dataflow
        Enhanced to support both regular and CI/CD dataflows using official Microsoft APIs
        
        Args:
            workspace_id: ID of the workspace
            dataflow_id: ID of the dataflow
            is_cicd_dataflow: Whether this is a CI/CD dataflow (auto-detected if None)
            
        Returns:
            Status of the latest refresh operation or None if not available
        """
        try:
            # Auto-detect if not specified
            if is_cicd_dataflow is None:
                is_cicd_dataflow = self._detect_dataflow_type(workspace_id, dataflow_id)
            
            if is_cicd_dataflow:
                # Official Microsoft Fabric API for CI/CD dataflow status
                endpoint = f"/v1/workspaces/{workspace_id}/items/{dataflow_id}/jobs/instances"
                
                try:
                    response = self.client.get(endpoint)
                    data = response.json()
                    
                    # The response should contain job instances
                    if 'value' in data and len(data['value']) > 0:
                        # Get the most recent job instance
                        latest_job = data['value'][0]  # Assuming sorted by most recent
                        status = latest_job.get("status", "Unknown")
                        self.logger.info(f"Latest CI/CD refresh status for dataflow {dataflow_id}: {status}")
                        return status
                    elif isinstance(data, list) and len(data) > 0:
                        latest_job = data[0]
                        status = latest_job.get("status", "Unknown")
                        self.logger.info(f"Latest CI/CD refresh status for dataflow {dataflow_id}: {status}")
                        return status
                    else:
                        self.logger.warning(f"No CI/CD job instances found for dataflow {dataflow_id}")
                        return None
                        
                except Exception as e:
                    self.logger.warning(f"CI/CD status endpoint failed, trying regular endpoint: {e}")
                    # Fallback to regular dataflow endpoint
                    is_cicd_dataflow = False
            
            if not is_cicd_dataflow:
                # Regular dataflow gen2 API
                endpoint = f"/v1.0/myorg/groups/{workspace_id}/dataflows/{dataflow_id}/transactions"
                
                response = self.client.get(endpoint)
                data = response.json()
                
                if 'value' in data and len(data['value']) > 0:
                    latest_transaction = data['value'][0]
                    status = latest_transaction.get("status", "Unknown")
                    self.logger.info(f"Latest regular refresh status for dataflow {dataflow_id}: {status}")
                    return status
                else:
                    self.logger.warning(f"No refresh transactions found for dataflow {dataflow_id}")
                    return None
                    
        except Exception as e:
            self.logger.error(f"Error checking refresh status: {e}")
            raise
            
    def wait_for_refresh_completion(self, workspace_id: str, dataflow_id: str, 
                                 timeout_minutes: int = 60, 
                                 check_interval_seconds: int = 30,
                                 is_cicd_dataflow: bool = None) -> str:
        """
        Wait for a dataflow refresh to complete
        Enhanced to support both regular and CI/CD dataflows
        
        Args:
            workspace_id: ID of the workspace
            dataflow_id: ID of the dataflow
            timeout_minutes: Maximum wait time in minutes
            check_interval_seconds: Interval between status checks in seconds
            is_cicd_dataflow: Whether this is a CI/CD dataflow (auto-detected if None)
            
        Returns:
            Final status of the refresh operation
        """
        self.logger.info(f"Waiting for dataflow {dataflow_id} refresh to complete (timeout: {timeout_minutes} minutes)")
        
        start_time = datetime.now()
        timeout = timedelta(minutes=timeout_minutes)
        
        while datetime.now() - start_time < timeout:
            # Close existing connection before potentially long wait
            # This is critical to avoid idle timeout issues
            self._close_connection()
            
            status = self.get_latest_refresh_status(workspace_id, dataflow_id, is_cicd_dataflow)
            
            if status in ["Success", "Failed", "Cancelled", "Completed", "Error", "Succeeded"]:
                self.logger.info(f"Dataflow refresh completed with status: {status}")
                return status
                
            self.logger.info(f"Current status: {status}, checking again in {check_interval_seconds} seconds")
            time.sleep(check_interval_seconds)
            
        self.logger.warning(f"Refresh timeout reached after {timeout_minutes} minutes")
        return "Timeout"

    def _parse_date(self, date_str: str) -> datetime:
        """
        Parse a date string into a datetime object
        
        Args:
            date_str: Date string in various formats
            
        Returns:
            Datetime object
        """
        try:
            # Try different formats
            for fmt in ['%Y-%m-%d', '%Y-%m-%d %H:%M:%S', '%Y-%m-%dT%H:%M:%S']:
                try:
                    return datetime.strptime(date_str, fmt)
                except ValueError:
                    continue
            
            # If all formats fail, raise exception
            raise ValueError(f"Unable to parse date string: {date_str}")
        except Exception as e:
            self.logger.error(f"Error parsing date: {e}")
            raise

    def _get_date_ranges(self, start_date: datetime, end_date: datetime, bucket_size_days: int) -> List[Tuple[datetime, datetime]]:
        """
        Split a date range into smaller buckets
        
        Args:
            start_date: Start date of the range
            end_date: End date of the range
            bucket_size_days: Size of each bucket in days
            
        Returns:
            List of (start_date, end_date) tuples for each bucket
        """
        date_ranges = []
        current_start = start_date
        
        while current_start < end_date:
            current_end = min(current_start + timedelta(days=bucket_size_days), end_date)
            
            # Set time to 23:59:59 for the end date of each bucket except the last one
            if current_end < end_date:
                current_end = datetime(current_end.year, current_end.month, current_end.day, 23, 59, 59)
                
            date_ranges.append((current_start, current_end))
            
            # Start the next bucket from the day after the current end
            current_start = current_end + timedelta(seconds=1)
        
        return date_ranges

    def execute_incremental_refresh(self,
                               workspace_id: str,
                               dataflow_id: str,
                               dataflow_name: str,
                               destination_table: str,
                               incremental_update_column: str,
                               initial_load_from_date: str = None,
                               bucket_size_in_days: int = 30,
                               reinitialize_dataflow: bool = False,
                               incrementally_update_last_n_days: int = None,
                               wait_for_completion: bool = True,
                               timeout_minutes: int = 120,
                               is_cicd_dataflow: bool = None,
                               bucket_retry_attempts: int = 3) -> Dict[str, Any]:
        """
        Execute a complete incremental refresh workflow following the specified logic
        Enhanced to support both regular and CI/CD dataflows

        Args:
            workspace_id: ID of the workspace
            dataflow_id: ID of the dataflow
            dataflow_name: Name or description of the dataflow
            destination_table: Name of the destination table
            incremental_update_column: Column used for incremental updates
            initial_load_from_date: Start date for the initial load (required for first load)
            bucket_size_in_days: Size of each refresh bucket in days
            reinitialize_dataflow: Whether to reinitialize the dataflow
            incrementally_update_last_n_days: Number of days to update incrementally
            wait_for_completion: Whether to wait for refresh completion
            timeout_minutes: Timeout when waiting for completion
            is_cicd_dataflow: Whether this is a CI/CD dataflow (auto-detected if None)
            bucket_retry_attempts: Number of times to retry a failed bucket before moving to next

        Returns:
            Dictionary containing execution results and statistics
        """
        # Ensure we have a fresh connection at the start
        self._ensure_connection()
        
        # Auto-detect dataflow type if not specified
        if is_cicd_dataflow is None:
            is_cicd_dataflow = self._detect_dataflow_type(workspace_id, dataflow_id)
        
        start_time = datetime.now()
        self.logger.info(f"Starting incremental refresh for {'CI/CD ' if is_cicd_dataflow else ''}dataflow {dataflow_id}")
        
        # Get yesterday's date at 23:59:59 as the default end date
        yesterday = datetime.now().replace(hour=0, minute=0, second=0, microsecond=0) - timedelta(days=1)
        yesterday_end = datetime(yesterday.year, yesterday.month, yesterday.day, 23, 59, 59)
        
        # Initialize statistics
        stats = {
            "dataflow_id": dataflow_id,
            "start_time": start_time,
            "buckets_refreshed": 0,
            "successful_refreshes": 0,
            "failed_refreshes": 0,
            "total_retry_attempts": 0,
            "current_status": "Started",
            "is_cicd_dataflow": is_cicd_dataflow
        }
        
        try:
            # Check if we need to reinitialize the dataflow
            if reinitialize_dataflow:
                self.logger.info(f"Reinitializing dataflow {dataflow_id} as requested")
                self.delete_incremental(dataflow_id)
                last_refresh_data = None
            else:
                # Get the last refresh record
                last_refresh_df = self.get_incremental(dataflow_id)
                last_refresh_data = last_refresh_df.iloc[0].to_dict() if last_refresh_df is not None else None
            
            # Case 1: No previous refresh or reinitializing dataflow
            if last_refresh_data is None:
                self.logger.info(f"No previous refresh found or reinitializing for dataflow {dataflow_id}")
                
                if not initial_load_from_date:
                    raise ValueError("initial_load_from_date is required for the first load")
                
                start_date = self._parse_date(initial_load_from_date)
                end_date = yesterday_end
                
                self.logger.info(f"Performing initial load from {start_date} to {end_date}")
                
                # Split the date range into buckets
                date_ranges = self._get_date_ranges(start_date, end_date, bucket_size_in_days)
                
                stats["date_ranges"] = len(date_ranges)
                stats["start_date"] = start_date
                stats["end_date"] = end_date
                
                for i, (range_start, range_end) in enumerate(date_ranges):
                    self.logger.info(f"Processing bucket {i+1}/{len(date_ranges)}: {range_start} to {range_end}")

                    # Retry loop for this bucket
                    bucket_success = False
                    for attempt in range(bucket_retry_attempts):
                        if attempt > 0:
                            self.logger.info(f"Retry attempt {attempt}/{bucket_retry_attempts-1} for bucket {i+1}/{len(date_ranges)}")
                            stats["total_retry_attempts"] += 1
                            # Exponential backoff: wait 30, 60, 120 seconds, etc.
                            wait_time = 30 * (2 ** (attempt - 1))
                            self.logger.info(f"Waiting {wait_time} seconds before retry...")
                            time.sleep(wait_time)

                        # Delete data in the range
                        self.delete_data(destination_table, incremental_update_column, range_start, range_end, dataflow_id)

                        # Insert a record with "Running" status
                        if i == 0 and attempt == 0:
                            self.insert_into_incremental(dataflow_id, workspace_id, dataflow_name, initial_load_from_date,
                                            bucket_size_in_days, incrementally_update_last_n_days, destination_table,
                                            incremental_update_column, "Running", range_start, range_end, is_cicd_dataflow)
                        else:
                            self.update_incremental(dataflow_id, "Running", range_start, range_end)

                        # Close connection before long-running dataflow operation
                        self._close_connection()

                        # Trigger dataflow refresh
                        self.refresh_dataflow(workspace_id, dataflow_id, dataflow_name, is_cicd_dataflow)

                        # Wait for completion if requested
                        if wait_for_completion:
                            # Already closed connection before this step
                            # Add a small buffer of 5 seconds just to make sure the status is updated
                            time.sleep(5)
                            status = self.wait_for_refresh_completion(workspace_id, dataflow_id, timeout_minutes, 30, is_cicd_dataflow)

                            # Ensure fresh connection for database operations
                            self._ensure_connection()

                            # Update status in the incremental table
                            self.update_incremental(dataflow_id, status, range_start, range_end)

                            if status in ["Success", "Succeeded", "Completed"]:
                                bucket_success = True
                                stats["successful_refreshes"] += 1
                                self.logger.info(f"Bucket {i+1}/{len(date_ranges)} completed successfully")
                                break  # Exit retry loop on success
                            else:
                                self.logger.warning(f"Refresh attempt {attempt+1}/{bucket_retry_attempts} failed for range {range_start} to {range_end} with status {status}")
                                if attempt == bucket_retry_attempts - 1:
                                    # Final attempt failed - raise exception to stop processing
                                    stats["failed_refreshes"] += 1
                                    stats["buckets_refreshed"] += 1
                                    error_msg = f"Bucket {i+1}/{len(date_ranges)} failed after {bucket_retry_attempts} attempts with status {status}. Range: {range_start} to {range_end}"
                                    self.logger.error(error_msg)
                                    raise RuntimeError(error_msg)
                        else:
                            self.logger.info("Not waiting for completion, continuing with next bucket")
                            bucket_success = True  # Assume success if not waiting
                            break

                    stats["buckets_refreshed"] += 1
            
            # Case 2: Previous refresh exists and we're not reinitializing
            else:
                last_status = last_refresh_data.get('status')
                last_range_start = last_refresh_data.get('range_start')
                last_range_end = last_refresh_data.get('range_end')
                
                # Use stored CI/CD flag if available, otherwise use detected value
                stored_cicd_flag = last_refresh_data.get('is_cicd_dataflow')
                if stored_cicd_flag is not None:
                    is_cicd_dataflow = bool(stored_cicd_flag)
                    stats["is_cicd_dataflow"] = is_cicd_dataflow
                
                self.logger.info(f"Previous refresh found with status '{last_status}' for range {last_range_start} to {last_range_end}")
                
                # Case 2a: Previous refresh was not successful, retry it
                if last_status not in ["Completed", "Success", "Successful", "Succeeded"]:
                    self.logger.info(f"Previous refresh was not successful, retrying for range {last_range_start} to {last_range_end}")

                    # Parse dates
                    range_start = last_range_start if isinstance(last_range_start, datetime) else self._parse_date(last_range_start)
                    range_end = last_range_end if isinstance(last_range_end, datetime) else self._parse_date(last_range_end)

                    # Retry loop for this bucket
                    bucket_success = False
                    for attempt in range(bucket_retry_attempts):
                        if attempt > 0:
                            self.logger.info(f"Retry attempt {attempt}/{bucket_retry_attempts-1} for previously failed bucket")
                            stats["total_retry_attempts"] += 1
                            # Exponential backoff: wait 30, 60, 120 seconds, etc.
                            wait_time = 30 * (2 ** (attempt - 1))
                            self.logger.info(f"Waiting {wait_time} seconds before retry...")
                            time.sleep(wait_time)

                        # Delete data in the range
                        self.delete_data(destination_table, incremental_update_column, range_start, range_end, dataflow_id)

                        # Update status to "Running"
                        self.update_incremental(dataflow_id, "Running", range_start, range_end)

                        # Close connection before long-running operation
                        self._close_connection()

                        # Trigger dataflow refresh
                        self.refresh_dataflow(workspace_id, dataflow_id, dataflow_name, is_cicd_dataflow)

                        # Wait for completion if requested
                        if wait_for_completion:
                            # Already closed connection above
                            # Add a small buffer of 5 seconds before checking status
                            time.sleep(5)
                            status = self.wait_for_refresh_completion(workspace_id, dataflow_id, timeout_minutes, 30, is_cicd_dataflow)

                            # Ensure fresh connection for database operations
                            self._ensure_connection()

                            self.update_incremental(dataflow_id, status, range_start, range_end)

                            if status in ["Success", "Succeeded", "Completed"]:
                                bucket_success = True
                                stats["successful_refreshes"] += 1
                                self.logger.info(f"Previously failed bucket completed successfully")
                                break  # Exit retry loop on success
                            else:
                                self.logger.warning(f"Retry attempt {attempt+1}/{bucket_retry_attempts} failed for range {range_start} to {range_end} with status {status}")
                                if attempt == bucket_retry_attempts - 1:
                                    # Final attempt failed - raise exception to stop processing
                                    stats["failed_refreshes"] += 1
                                    stats["buckets_refreshed"] += 1
                                    error_msg = f"Previously failed bucket failed after {bucket_retry_attempts} attempts with status {status}. Range: {range_start} to {range_end}"
                                    self.logger.error(error_msg)
                                    raise RuntimeError(error_msg)
                        else:
                            bucket_success = True  # Assume success if not waiting
                            break

                    stats["buckets_refreshed"] += 1
                
                # Case 2b: Previous refresh was successful, continue with next incremental update
                else:
                    self.logger.info("Previous refresh was successful, calculating next incremental update range")
                    
                    # Parse the last range end date
                    last_end_date = last_range_end if isinstance(last_range_end, datetime) else self._parse_date(last_range_end)

                    # End date is yesterday
                    range_end = yesterday_end

                    # If incrementally_update_last_n_days is provided, use it to potentially overlap with previous data
                    if incrementally_update_last_n_days:
                        possible_start = yesterday - timedelta(days=incrementally_update_last_n_days)
                        # Use the minimum of the two possible start dates to ensure there are no gaps
                        # If last_end_date + 1 second is earlier, use that to continue from where we left off
                        # If possible_start is earlier, use that to ensure we include the last N days
                        range_start = min(last_end_date + timedelta(seconds=1), possible_start)
                        self.logger.info(f"Using start date {range_start} (minimum of last_end_date+1 second and last {incrementally_update_last_n_days} days)")
                    else:
                        # Start from the day after the last end date
                        range_start = last_end_date + timedelta(seconds=1)
                        self.logger.info(f"Using start date {range_start} (continuing from last end date)")

                    # Only proceed if there's actual data to refresh
                    if range_start < range_end:
                        self.logger.info(f"Performing incremental update from {range_start} to {range_end}")

                        # Split the date range into buckets
                        date_ranges = self._get_date_ranges(range_start, range_end, bucket_size_in_days)

                        stats["date_ranges"] = len(date_ranges)
                        stats["start_date"] = range_start
                        stats["end_date"] = range_end
                        
                        for i, (bucket_start, bucket_end) in enumerate(date_ranges):
                            self.logger.info(f"Processing bucket {i+1}/{len(date_ranges)}: {bucket_start} to {bucket_end}")

                            # Retry loop for this bucket
                            bucket_success = False
                            for attempt in range(bucket_retry_attempts):
                                if attempt > 0:
                                    self.logger.info(f"Retry attempt {attempt}/{bucket_retry_attempts-1} for bucket {i+1}/{len(date_ranges)}")
                                    stats["total_retry_attempts"] += 1
                                    # Exponential backoff: wait 30, 60, 120 seconds, etc.
                                    wait_time = 30 * (2 ** (attempt - 1))
                                    self.logger.info(f"Waiting {wait_time} seconds before retry...")
                                    time.sleep(wait_time)

                                # Delete data in the range
                                self.delete_data(destination_table, incremental_update_column, bucket_start, bucket_end, dataflow_id)

                                # Insert a record with "Running" status
                                self.update_incremental(dataflow_id, "Running", bucket_start, bucket_end)

                                # Close connection before long-running operation
                                self._close_connection()

                                # Trigger dataflow refresh
                                self.refresh_dataflow(workspace_id, dataflow_id, dataflow_name, is_cicd_dataflow)

                                # Wait for completion if requested
                                if wait_for_completion:
                                    # Already closed connection above
                                    # Add a small buffer of 5 seconds before checking status
                                    time.sleep(5)
                                    status = self.wait_for_refresh_completion(workspace_id, dataflow_id, timeout_minutes, 30, is_cicd_dataflow)

                                    # Ensure fresh connection for database operations
                                    self._ensure_connection()

                                    # Update status in the incremental table
                                    self.update_incremental(dataflow_id, status, bucket_start, bucket_end)

                                    if status in ["Success", "Succeeded", "Completed"]:
                                        bucket_success = True
                                        stats["successful_refreshes"] += 1
                                        self.logger.info(f"Bucket {i+1}/{len(date_ranges)} completed successfully")
                                        break  # Exit retry loop on success
                                    else:
                                        self.logger.warning(f"Refresh attempt {attempt+1}/{bucket_retry_attempts} failed for range {bucket_start} to {bucket_end} with status {status}")
                                        if attempt == bucket_retry_attempts - 1:
                                            # Final attempt failed - raise exception to stop processing
                                            stats["failed_refreshes"] += 1
                                            stats["buckets_refreshed"] += 1
                                            error_msg = f"Bucket {i+1}/{len(date_ranges)} failed after {bucket_retry_attempts} attempts with status {status}. Range: {bucket_start} to {bucket_end}"
                                            self.logger.error(error_msg)
                                            raise RuntimeError(error_msg)
                                else:
                                    self.logger.info("Not waiting for completion, continuing with next bucket")
                                    bucket_success = True  # Assume success if not waiting
                                    break

                            stats["buckets_refreshed"] += 1
                    else:
                        self.logger.info(f"No new data to refresh. Last refresh end date {last_end_date} is after or equal to start date {range_start}")
                        stats["current_status"] = "No new data to refresh"
            
            # Calculate total duration
            stats["end_time"] = datetime.now()
            stats["duration_seconds"] = (stats["end_time"] - start_time).total_seconds()
            stats["current_status"] = "Completed" if stats["failed_refreshes"] == 0 else f"Completed with {stats['failed_refreshes']} failures"
            
            self.logger.info(f"Incremental refresh completed for {'CI/CD ' if is_cicd_dataflow else ''}dataflow {dataflow_id}. "
                          f"Total buckets: {stats['buckets_refreshed']}, "
                          f"Successful: {stats['successful_refreshes']}, "
                          f"Failed: {stats['failed_refreshes']}")
            
            return stats
            
        except Exception as e:
            error_message = f"Error executing incremental refresh: {str(e)}"
            self.logger.error(error_message)
            
            # Update stats with error information
            stats["end_time"] = datetime.now()
            stats["duration_seconds"] = (stats["end_time"] - start_time).total_seconds()
            stats["current_status"] = f"Failed: {str(e)}"
            stats["error"] = str(e)
            
            # Ensure we close the connection on error
            self._close_connection()
            
            return stats
        finally:
            # Make sure to close the connection when done
            self._close_connection()

# Create the power bi rest client
client = fabric.PowerBIRestClient()

# Create the dataflow refresher object
dataflow_refresher = DataflowRefresher(
    client, 
    CONNECTION_ARTIFACT, 
    CONNECTION_ARTIFACT_ID, 
    CONNECTION_ARTIFACT_TYPE,
    SCHEMA, INCREMENTAL_TABLE, 
    logging.INFO)

# Incrementally refresh the dataflow
try:
    result = dataflow_refresher.execute_incremental_refresh(
        workspace_id,
        dataflow_id,
        dataflow_name,
        destination_table,
        incremental_update_column,
        initial_load_from_date,
        bucket_size_in_days,
        reinitialize_dataflow,
        incrementally_update_last_n_days,
        is_cicd_dataflow=is_cicd_dataflow,
        bucket_retry_attempts=bucket_retry_attempts
    )

    # Print the execution results
    print(f"Dataflow refresh execution completed:")
    print(f"Status: {result.get('current_status')}")
    print(f"Total buckets processed: {result.get('buckets_refreshed', 0)}")
    print(f"Successful refreshes: {result.get('successful_refreshes', 0)}")
    print(f"Failed refreshes: {result.get('failed_refreshes', 0)}")
    print(f"Total retry attempts: {result.get('total_retry_attempts', 0)}")
    print(f"Duration: {result.get('duration_seconds', 0):.2f} seconds")
    print(f"Dataflow type: {'CI/CD' if result.get('is_cicd_dataflow') else 'Regular'}")

    # Exit with success code
    import sys
    sys.exit(0)

except RuntimeError as e:
    # Bucket refresh failed after all retries
    print(f"\n{'='*80}")
    print(f"DATAFLOW REFRESH FAILED")
    print(f"{'='*80}")
    print(f"Error: {str(e)}")
    print(f"\nThe dataflow refresh has been terminated due to bucket failure after all retry attempts.")
    print(f"Please check the logs above for detailed error information.")
    print(f"{'='*80}\n")

    # Exit with failure code
    import sys
    sys.exit(1)

except Exception as e:
    # Other unexpected errors
    print(f"\n{'='*80}")
    print(f"DATAFLOW REFRESH ERROR")
    print(f"{'='*80}")
    print(f"An unexpected error occurred: {str(e)}")
    print(f"{'='*80}\n")

    # Exit with failure code
    import sys
    sys.exit(1)

In [ ]:
# Update the refresh_dataflow method
def refresh_dataflow(self, workspace_id: str, dataflow_id: str) -> Dict[str, Any]:
    """
    Refresh dataflow Gen2 and return the response
    
    Args:
        workspace_id: ID of the workspace
        dataflow_id: ID of the dataflow
        
    Returns:
        Dictionary containing the response from the refresh API
    """
    try:
        # For Dataflow Gen2 CI/CD objects, use Fabric REST API
        response = self.client.post(f"/v1/workspaces/{workspace_id}/items/{dataflow_id}/jobs/instances?jobType=Pipeline", json={})
        self.logger.info(f"Successfully triggered refresh for dataflow Gen2 {dataflow_id}")
        return response
    except Exception as e:
        self.logger.error(f"Error triggering refresh: {e}")
        raise

def get_latest_refresh_status(self, workspace_id: str, dataflow_id: str) -> Optional[str]:
    """
    Get refresh status of a dataflow Gen2
    
    Args:
        workspace_id: ID of the workspace
        dataflow_id: ID of the dataflow
        
    Returns:
        Status of the latest refresh operation or None if not available
    """
    try:
        # For Dataflow Gen2 CI/CD objects, use Fabric REST API
        response = self.client.get(f"/v1/workspaces/{workspace_id}/items/{dataflow_id}/jobs/instances")
        instances = response.json()['value']
        if instances and len(instances) > 0:
            latest_instance = instances[0]
            status = latest_instance.get("status", "Unknown")
            self.logger.info(f"Latest refresh status for dataflow Gen2 {dataflow_id}: {status}")
            return status
        else:
            self.logger.warning(f"No refresh instances found for dataflow Gen2 {dataflow_id}")
            return None
    except Exception as e:
        self.logger.error(f"Error checking refresh status: {e}")
        raise